In [202]:
from collections import defaultdict
import requests
from lxml import etree
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [365]:
reddit = defaultdict(lambda:defaultdict(lambda:0))

In [366]:
url = "https://www.reddit.com/subreddits/"
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, "html.parser")


In [367]:
subreddit = soup.find_all('p', 'titlerow')
des = soup.find_all('div', 'md')[1:]
sub = soup.find_all('span', 'likes')


In [368]:
for i in range(len(subreddit)): 
    reddit[subreddit[i].a.text]['description'] = des[i].p.text
    reddit[subreddit[i].a.text]['subscribers'] = sub[i].find('span', 'number').text
    

In [369]:
next_b = soup.find('span', 'next-button')
while(next_b):
    url = next_b.a['href']
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text, "html.parser")
    subreddit = soup.find_all('p', 'titlerow')
    des = soup.find_all('div', 'md')[1:]
    sub = soup.find_all('span', 'likes')
    for i in range(len(subreddit)):
        reddit[subreddit[i].a.text]['description'] = des[i].p.text if des[i].p else ''
        reddit[subreddit[i].a.text]['subscribers'] = sub[i].find('span', 'number').text
    next_b = soup.find('span', 'next-button')

IndexError: list index out of range

In [370]:
for i in reddit:
    reddit[i]['subscribers'] = int(reddit[i]['subscribers'].replace(',',''))

In [371]:
reddit = sorted(reddit.items(), key = lambda x:x[1]['subscribers'], reverse=True)

In [386]:
import csv


with open('output.csv', 'w') as output:
    writer = csv.writer(output)
    writer.writerow(['subreddit', 'description', 'subscribers'])
    for i in range(len(reddit)):
        writer.writerow([reddit[i][0], reddit[i][1]['subscribers'], reddit[i][1]['description']])